<a href="https://colab.research.google.com/github/gokul-krishna-r/dm-logistic-regression/blob/main/dm_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DM Assignment

Submitted by



```
Gokul Krishna R
CS A
Roll no: 41
20220040
```





---
**Data Preprocessing**
---

Data preprocessing is the initial phase in data analysis and machine learning, where the raw data is cleaned, transformed, and organized to make it suitable for further analysis or model trainings. The preprocessing done here are

*  **Feature extraction** selects relevant information and creates new features to capture essential patterns
*   **Missing value repalcement** handling addresses data points with missing information, using methods like imputation or removal

*   **Scaling** transforms numerical features to have similar scales, improving algorithm performance.
*  **Categorical encoding** converts non-numeric categories into numerical values

In [28]:
import pandas as pd
import numpy as np

In [29]:
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
#to plot figures
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import matplotlib as mpl

drive.mount('/content/dataset/')

Drive already mounted at /content/dataset/; to attempt to forcibly remount, call drive.mount("/content/dataset/", force_remount=True).


In [23]:
train=pd.read_csv('/content/dataset/MyDrive/Colab Notebooks/train.csv')
test=pd.read_csv('/content/dataset/MyDrive/Colab Notebooks/test.csv')
#add column in each dataset
train["Type"]='train'
test["Type"]='test'
#merge data
data=train.append(test,sort=False)

#show initial 5 rows
data.head()


<ipython-input-23-f798944b01d4>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data=train.append(test,sort=False)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Type
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train


**Feature extraction**

The process of selecting or creating relevant and informative characteristics (features) from raw data to be used as input for machine learning models or data analysis. The main goal of feature extraction is to reduce the dimensionality of the data and to represent it in a more meaningful way.

In [24]:
for name in data["Name"]:
  data['Title']=data["Name"].str.extract('([A-za-z]+)\.',expand=True)

#replace uncommon titles
mapping = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Other', 'Col': 'Other', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Other', 'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
data.replace({"Title":mapping},inplace=True)

data["Family_Size"]=(data["Parch"]+data["SibSp"]).astype(int)
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Type,Title,Family_Size
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train,Mr,1
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train,Mrs,1
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train,Miss,0
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train,Mrs,1
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train,Mr,0


**Missing values**

are data points that are not available or not recorded for some reasons. Dealing with missing values is essential to ensure that the data is valid and complete before using it for analysis or modeling. Here the missing values are replaced by mean substitution.

In [25]:
data["Age"].fillna(data.groupby(["Sex","Pclass"])["Age"].transform("mean"),inplace=True)

data.loc[61,'Embarked']='S'
data.loc[829,'Embarked']='S'

data["Fare"].fillna(data["Fare"].mean(),inplace=True)

**Categorical encoding**

is the process of converting categorical data, which represents qualitative or non-numeric variables, into numerical values so that they can be used as inputs for machine learning algorithms.

In [26]:
data.drop(["Cabin",'Name',"Ticket","PassengerId","SibSp","Parch"],axis=1,inplace=True)

data["Sex"]=data["Sex"].astype('category')
data["Sex"]=data["Sex"].cat.codes

categorical=["Embarked","Title","Pclass"]
for cat in categorical:
  data=pd.concat([data,pd.get_dummies(data[cat],prefix=cat)],axis=1)
  del data[cat]
data.head()

,Survived,Sex,Age,Fare,Type,Family_Size,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Title_Royal,Pclass_1,Pclass_2,Pclass_3
0,0.0,1,22.0,7.2500,train,1,0,0,1,0,0,1,0,0,0,0,0,1
1,1.0,0,38.0,71.2833,train,1,1,0,0,0,0,0,1,0,0,1,0,0
2,1.0,0,26.0,7.9250,train,0,0,0,1,0,1,0,0,0,0,0,0,1
3,1.0,0,35.0,53.1000,train,1,0,0,1,0,0,0,1,0,0,1,0,0
4,0.0,1,35.0,8.0500,train,0,0,0,1,0,0,1,0,0,0,0,0,1


**Scaling**

involves transforming the features of a dataset to bring them to a similar scale or range. If the features are on vastly different scales, some algorithms might give more importance to features with larger values, leading to biased or suboptimal results.

In [37]:
#Scaling
continous=["Age","Fare","Family_Size"]
scaler=StandardScaler()
for val in continous:
  data[val]=data[val].astype("float64")
  data[val]=scaler.fit_transform(data[val].values.reshape(-1,1))
data.head()

,Survived,Sex,Age,Fare,Type,Family_Size,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Title_Royal,Pclass_1,Pclass_2,Pclass_3
0,0.0,1,-0.560331,-0.503595,train,0.073352,0,0,1,0,0,1,0,0,0,0,0,1
1,1.0,0,0.655107,0.734503,train,0.073352,1,0,0,0,0,0,1,0,0,1,0,0
2,1.0,0,-0.256471,-0.490544,train,-0.558346,0,0,1,0,1,0,0,0,0,0,0,1
3,1.0,0,0.427212,0.382925,train,0.073352,0,0,1,0,0,0,1,0,0,1,0,0
4,0.0,1,0.427212,-0.488127,train,-0.558346,0,0,1,0,0,1,0,0,0,0,0,1


**Training and test datasets separation**

separates the training and test datasets from the previous merged dataset, and converts the data to the appropriate format for model training

In [46]:
X_train = data[data['Type'] == 'train'].drop(columns = ['Type', 'Survived'])
y_train= data[data['Type'] == 'train']['Survived']

X_test = data[data['Type'] == 'test'].drop(columns = ['Type', 'Survived'])
X_test.head()

,Sex,Age,Fare,Family_Size,Embarked_C,Embarked_Q,Embarked_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Title_Royal,Pclass_1,Pclass_2,Pclass_3
0,1,0.389230,-0.492396,-0.558346,0,1,0,0,0,1,0,0,0,0,0,1
1,0,1.338790,-0.508429,0.073352,0,0,1,0,0,0,1,0,0,0,0,1
2,1,2.478263,-0.456465,-0.558346,0,1,0,0,0,1,0,0,0,0,1,0
3,1,-0.180507,-0.476284,-0.558346,0,0,1,0,0,1,0,0,0,0,0,1
4,0,-0.560331,-0.406194,0.705051,0,0,1,0,0,0,1,0,0,0,0,1



---
**Data Classification**
---

**Regression**

It is a statistical method used for predicting numerical values based on input features. The main goal of regression is to establish a relationship between the independent variables (input features) and the dependent variable (output or target variable) to make predictions. In essence, it tries to find a mathematical function that best fits the data.

The classification model used here is Logistic regression

**Logisitic Regression**

It is used to predict the probability that an instance belongs to a particular class. The output of logistic regression is a value between 0 and 1, representing the probability of the positive class.

**Steps**

1. Upload dataset
2. Split the dataset- x (dependent variables), y(independent variables)
3. Learning -> Output: Model
4. Train the model
5. Evaluate the model using metrics(['accuracy'])

**Detailed Explanation**

Logistic regression model has been implemented.

Titanic dataset that contains features related to voyagers in Titanic and a target variable 'Survived' which indicates whether a person has survived (1) or not (0) is loaded using pandas

The dataset is split into two parts: the input features (X) and the target variable (y). The X contains all the columns except for the 'Survived' column, and y contains only the 'Survived' column (target variable). The data is further split into training and testing sets. The training set will be used to train the the model, and the testing set will be used to evaluate the model.

Importing LogisticRegression from sklearn.linear_model provides us with Linear regression model to start training. The max_iter parameter is set to 200, specifying the maximum number of iterations allowed for the algorithm to converge. Fit method is used to train the model on the training data.

The accuracy of the model is evaluated by making it perform predictions on the test data using the accuracy_score function from scikit-learn. It compares the predicted outcomes (y_pred) with the actual outcomes from the testing set (y_train) and calculates the accuracy of the model.

In [45]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create a logistic regression model
model = LogisticRegression(max_iter=200) #specifies the number of iterations for the algorithm to converge

# Train the model on the training data (input features -x and output - y(dependent variable))
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_train)

# Evaluate the model
accuracy = accuracy_score(y_train, y_pred)
# Print the evaluation results
print('Accuracy:', accuracy)

Accuracy: 0.8338945005611672


 **Making Use of the Model**

 A new data point is created as a DataFrame. The predict method of the trained model is used to predict whether this new data point belongs to the survived class (1) or the not survived class(0).

In [49]:
new_data = pd.DataFrame([[0, 0.200230,-0.492396,-0.558346,10,0,1,0,0,1,1,1,0,0,0,1]])
prediction = model.predict(new_data)
if prediction == 0:
  print("Result: Not Survived")
else:
  print("Result: Survived")

Result: Survived


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
